# Build a ParFlow run from a template

>>> TODO: Some intro about parflow and the role that runscripts play in a simulation here?

The subsettools package comes with a collection of runscripts that address a variety of common configurations for ParFlow. You may use these as a template for a ParFlow run that most closely meets the specifications of the model you are trying to build. In this tutorial we are going to show how to get one of the provided template runscripts, modify its keys to match our subsetting domain and our filename/directory structure and finally build a ParFlow run object from our edited runscript.

### 1.  Setup 

In all examples you will need to import the following packages and register your pin in order to have access to the HydroData datasets. <<< for this workbook registering is probably not necessary, but we can keep it anyway?

Refer to the [getting started](https://hydroframesubsettools.readthedocs.io/en/latest/getting_started.html) instructions for creating your pin if you have not done this already.

In [1]:
from subsettools.subsettools import (
    huc_to_ij, 
    latlon_to_ij, 
    subset_press_init, 
    edit_runscript_for_subset,
    change_filename_values,
    dist_run,
)
from subsettools.datasets import get_template_runscript
from hf_hydrodata import grid, gridded
from parflow import Run
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

gridded.register_api_pin("your_email", "your_pin")

### 2. Get a template runscript

In order to run ParFlow, you must have a model object in the form of a .pfidb or .yaml file (link?). These files can be generated initially from a python script using pftools keys. You can read more about pftools and functions for manipulating runscripts [here](https://parflow.readthedocs.io/en/latest/python/run_script.html).

We recommend a user select a template runscript corresponding to following three guidelines:

    1. Select a source data domain you wish to subset from: a. CONUS1 b. CONUS2

    2. Select an input file type: a. Solid file (if you are providing a HUC or HUC list) b. A box domain (you are providing lat/lon coordinates)

    3. Select a template corresponding to your planned way to run ParFlow: a. Transient run, fully coupled to the climate model CLM b. Model Initialization running ParFlow on its own with a longterm recharge (PmE) mask.

We provide 8 template runscripts which correspond to all unique combinations of the above three guidelines. These template runscripts have values set to the values used to run simulations of CONUS1 or CONUS2 in the past.

To get a template runscript provided with the package, you should use the `get_template_runscript` function (API reference [here](https://hydroframesubsettools.readthedocs.io/en/edit-docs/autoapi/subsettools/datasets/index.html#subsettools.datasets.get_template_runscript)). The function will get the correct template runscript based on your choices, write it to your chosen directory, and return the path to the file written.

For example, to get the template for a ParFlow-CLM coupled run on the CONUS1 grid with a solid input file, you can do:

In [2]:
template_run = get_template_runscript(
    grid="conus1", 
    mode="transient", 
    input_file_type="solid",
    write_dir="/home/ga6/subsettools_example",
)

We can create a ParFlow run object from a runscript file with the `from_definition` method of the ParFlow `Run` class. We can then use standard python attribute notation to explore the run keys and see their values:

In [3]:
run = Run.from_definition(template_run)
# Let's check that our input is a solid file:
print("Input file type:", run.GeomInput.domaininput.InputType)

# >>>> TODO: what other keys to explore at this stage?

Input file type: SolidFile


### 3. Define your area of interest

In order to custom our runscript to our subsetting domain, we first need to calculate the grid bounds. We show how to do that below.

We use i,j indices in order to define the subset the national files that you would like to extract.  The [`latlon_to_ij`](https://hydroframesubsettools.readthedocs.io/en/latest/autoapi/subsettools/subsettools/index.html#subsettools.subsettools.latlon_to_ij) function translates a bounding box in lat-lon  coordinates bounds to i,j indices in whatever grid system we select. It returns a tuple `(imin, jmin, imax, jmax)` of grid indices that define a bounding box containing our region (or point) of interest (Note: `(imin, jmin, imax, jmax)` are the west, south, east and north boundaries respectively).

Here we will show how to define a subset extent for (1) a single point of interest, (2) a user specified bounding box, and (3) a bounding box that surrounds a user specified HUC. 

**IMPORTANT NOTE**: *The i,j indices found in this step are based on whatever grid you select (e.g. `conus1` or `conus2`). Its very important that the grid you use in this step is the same as the grid that the data files (static input and forcing) you are subsetting are in or you will end up subsetting a different location than you expect.  The grids are shown below and described in [Yang et al 2023](https://www.sciencedirect.com/science/article/pii/S0022169423012362)* 

![CONUS domains](CONUS1_2_domain.jpg)

#### 3.1 Defining bounds to extract data for a single point
To extract data for a single point we use the same bounding box function as we would to extract a larger domain but just repeat the point values as the upper and lower bounds.

In [5]:
lat = 39.8379
lon = -74.3791
# Since we want to subset only a single location, both lat-lon bounds are defined by this point:
latlon_bounds = ([lat, lon],[lat, lon])
ij_column_bounds = latlon_to_ij(latlon_bounds=latlon_bounds, grid="conus2")
print(f"bounding box: {ij_column_bounds}")

bounding box: (4057, 1915, 4057, 1915)


#### 3.2 Defining bounds for a box defined by lat-lon bounds
To extract a bounding box, provide the upper and lower latitude and longitude bounds respectively for the area of interest as well as the grid system that you would like to use. 

In [6]:
ij_box_bounds = latlon_to_ij(latlon_bounds=[[37.91, -91.43], [37.34, -90.63]], grid="conus1")
print(f"bounding box: {ij_box_bounds}")

bounding box: (2285, 436, 2358, 495)


#### 3.3 Defining bounds for a HUC watershed
The subsettools [`huc_to_ij`](https://hydroframesubsettools.readthedocs.io/en/latest/autoapi/subsettools/subsettools/index.html#subsettools.subsettools.huc_to_ij) function returns a tuple `(imin, jmin, imax, jmax)` of grid indices that define a bounding box containing any HUC. You can provide 2, 4, 6, 8 or 10-digit HUCs.  For help finding your HUC you can refer to the [USGS HUC picker](https://water.usgs.gov/wsc/map_index.html).

In [7]:
ij_huc_bounds = huc_to_ij(huc_list=["14050002"], grid="conus2")
print(f"bounding box: {ij_huc_bounds}")

bounding box: (1225, 1738, 1347, 1811)


### 4. Modify a template runscript for  a subset domain 
We will now use the `edit_runscript_for_subset` function (API reference [here](https://hydroframesubsettools.readthedocs.io/en/edit-docs/autoapi/subsettools/subsettools/index.html#subsettools.subsettools.edit_runscript_for_subset)) to tailor our runscript to our subsetting domain. We will pass the function the grid bounds we calculated (we will pass `ij_huc_bounds` since we chose a runscript for a solid file), the path to our template runscript, and a new name for our run. We can also pass a path to our forcing directory, if we have subset and saved our forcing data already (for that take a look at our example on subsetting forcing data). The function will return a path to the new runscript that will be created.

**NOTE:** *If you don't provide a write_dir argument, the new runscript is going to be written in the directory or the original runscript. The filename depends on the runname, so if you also don't change the runname the original runscript file will be overwritten.* <<< Is this clear?

In [8]:
runscript_path = edit_runscript_for_subset(
    ij_bounds=ij_huc_bounds,
    runscript_path=template_run,
    runname="my_new_conus2_run",
)

New runname: my_new_conus2_run provided, a new yaml file will be created
No forcing directory provided, run.Solver.CLM.MetFilePath key not set
ComputationalGrid.NY set to 73 and NX to 122
GeomInput.domaininput.InputType detected as SolidFile, no additional keys to change for subset
Updated runscript written to /home/ga6/subsettools_example


In [9]:
# we can explore the mofified keys as before, by created a ParFlow run object from the new runscript:
run = Run.from_definition(runscript_path)
print("New runname:", run.get_name())
print("New grid nx, ny:", run.ComputationalGrid.NX, run.ComputationalGrid.NY)

New runname: my_new_conus2_run
New grid nx, ny: 122 73


### 5. Modify file paths for inputs in the runscript

We can use the subsettools `change_filename_values` function  (API reference [here](https://hydroframesubsettools.readthedocs.io/en/edit-docs/autoapi/subsettools/subsettools/index.html#subsettools.subsettools.change_filename_values)) to modify the file paths for various ParFlow inputs in our runscript. For example, we can use the subset_press_init` function to get initial pressure data. This function returns a path to the file written and we will modify our runscript so that ParFlow can find that file.

>>> This step is a little akward. In order to use the `change_filename_values` function I have to assume that files exist. I think the easiest is to use either subset_static or subset_press_init and grab the filepaths from their return values?



**NOTE:** *You should provide the filename, not the entire path to the file. We suggest using the Python Standard Library [`os.path.basename`](https://docs.python.org/3/library/os.path.html#os.path.basename) function to extract the filename from the path.*

In [ ]:
# get and save the initial pressure data <<< this won't run until we put a dataset
init_press_filepath = subset_press_init(
    ij_huc_bounds,
### which dataset would I use for conus2? for conus1 it would be conus1_baseline_mod
    dataset="some_dataset",
    date="2005-12-15",
    write_dir="/home/ga6/subsettools_example",
    time_zone="EST",
)

# change the initial pressure file path in the runscript:
filename = os.path.basename(press_init_filepath)
runscript_path = change_filename_values(
    runscript_path=runscript_path,
    init_press=init_press,
)

### 6. Change the processor topology

>>> TODO: something about parallel computing / how parflow distributes the simulation here?

We will use the subsettools `dist_run` function (API reference [here](https://hydroframesubsettools.readthedocs.io/en/edit-docs/autoapi/subsettools/subsettools/index.html#subsettools.subsettools.dist_run)) to distribute our inputs to our chosen processor topology.

>>> This is also akward like step 5, as it assumes we already have a directory with static and/or forcing inputs. I feel like this tutorial is trying to emulate the example notebook, but without the necessary data to make the code run. Perhaps we can improve the narrative of the notebook instead?